In [1]:
# pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
# pip install -qU "langchain[aws]"
# pip install python-dotenv
# pip install -qU langchain-aws
# pip install -qU langchain-core
# ollama pull llama3.1
# pip install langchain-ollama



# Investigate:
# pip install unstructured
# pip install "unstructured[pdf]"

In [39]:
import sys
print(sys.executable)

/Users/ktejwani/Local_Ollama_RAG/llama_venv/bin/python


### Set Up Credentials and Envitoment Vars

In [17]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')
os.environ['AWS_REGION'] = os.getenv('AWS_REGION')

import os
os.environ['AWS_CA_BUNDLE'] = '/Users/ktejwani/.certs/Zscaler.pem'


### Initialize Gen and Embed LLM

In [16]:
from langchain_aws import BedrockEmbeddings

import boto3
import os
session = boto3.Session()
bedrock_client = session.client(
    service_name='bedrock-runtime',
    region_name=os.getenv('AWS_REGION'),
    verify=False
)

from langchain.chat_models import init_chat_model

llm = init_chat_model("anthropic.claude-3-5-sonnet-20240620-v1:0", model_provider="bedrock_converse")


import boto3
from botocore.config import Config
from langchain_aws import BedrockEmbeddings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
insecure_config = Config(
    connect_timeout=5,
    read_timeout=5,
    retries={'max_attempts': 10},
    signature_version='v4'
)
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-west-2",
    config=insecure_config,
    verify=False
)
embeddings = BedrockEmbeddings(
    client=bedrock_client,
    model_id="amazon.titan-embed-text-v2:0"
)



##### Local work around(NO API Calls)

In [ ]:
# from langchain_ollama import OllamaEmbeddings

# embeddings = OllamaEmbeddings(
#     model="nomic-embed-text"
# )


### Loading PDF Documents from a Directory :)

In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader

# Path to your data directory
data_directory = "data"
docs = []

print(f"Loading documents from: {data_directory}")

# Loop through each file in the directory
for filename in os.listdir(data_directory):
    if filename.endswith(".pdf"):
        file_path = os.path.join(data_directory, filename)
        print(f"--> Loading {filename}...")
        try:
            # Use PyPDFLoader for each file
            loader = PyPDFLoader(file_path)
            # Extend the main docs list with the pages from the current PDF
            docs.extend(loader.load())
            print(f"    ...success!")
        except Exception as e:
            # If a file fails to load, print an error and continue
            print(f"    ...ERROR loading {filename}: {e}")

print(f"\nSuccessfully loaded a total of {len(docs)} pages from all documents.")

Loading documents from: data
--> Loading tesla 10q.pdf...
    ...success!
--> Loading nvidia 10q.pdf...
    ...success!
--> Loading May Apple 10q.pdf...
    ...success!

Successfully loaded a total of 111 pages from all documents.


### Splitting documents via character

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)

all_splits = text_splitter.split_documents(docs)
print(f"Split all documents into {len(all_splits)} sub-documents.")


Split all documents into 469 sub-documents.


### Initilize In-Memory Vector Store

In [2]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)


### Embedding in a vector store

In [15]:
#Quick embed test
# # Example usage (replace with Kaartik's actual vector_store and all_splits)
example_text = ["This is a test sentence for embedding.", "Another sentence to embed."]
vectors = embeddings.embed_documents(example_text)
print(f"Generated {len(vectors)} vectors, each with dimension {len(vectors[0])}")

Generated 2 vectors, each with dimension 1024


In [ ]:
# document_ids = vector_store.add_documents(documents=all_splits)

# Assuming vector_store and all_splits are defined
try:
    document_ids = vector_store.add_documents(documents=all_splits)
    print("Documents successfully added to vector store (SSL verification bypassed).")
    # print(f"Added document IDs: {document_ids}")
except Exception as e:
    print(f"An error occurred during document addition: {e}")



Documents successfully added to vector store (SSL verification bypassed).
Added document IDs: ['569b6859-2652-47bf-bf3e-67703f7acf5d', '05b17e73-896b-4ffb-bb0f-8bd5c9d36d08', '9e63684a-2b4a-4b1f-a6d5-610ad820a9a1', 'e7b80938-e0a1-4f2b-a845-9963c5db6cbc', 'a834eb7c-4d73-45e8-a447-aad230d6b887', 'a478259d-1e58-4420-95ff-9fc8dc2f641a', 'cb284cf3-106f-4c00-acd1-59a38996ffe4', 'b84c4499-9509-4a35-b56b-f776b0b2d8f7', 'f78def7b-dd0a-4ff8-afba-76865680da33', '85e53a5c-4fc0-4167-859d-b2430e4cf714', 'a1254f01-1549-44ed-aa7f-4ab1dbe2f11f', 'a70a7383-f2f9-44d0-acae-cb6b2b298530', 'c081b3d9-6155-4828-8736-d956fb6e3669', 'd5980ce7-c1f6-40cc-a705-c722aea507de', 'a3b01ec7-eb61-4839-a910-f68f2842a693', 'a199a07e-46f7-4820-8ba3-16a77f22498f', '6b00ebcb-e5d9-48fe-9d26-2627328c1da1', '4fc4c681-6891-4aac-9d0b-8a6041bd61fe', '38612347-f537-4c1f-b2a3-c04c1056f98e', 'cca23ceb-414d-456d-83e5-c6943f35f003', 'fd5b8f0b-a0bd-465b-8485-f68dad10be0a', 'c8cc4732-94cb-4d63-9f5a-a6483e5a2e41', 'a560c421-4f92-4da3-a24c-

#### Testing retrieval

In [ ]:
#What does apple sell? <-- test question
question = "What does apple sell??"

print(f"\nSearching for documents relevant to: '{question}'")
# Use the vector store to find similar documents
# It automatically embeds the user's question and compares it to the chunk vectors.
# Default 4 chunks(k=4)
results = vector_store.similarity_search(query=question)


#  Inspect the Results ---

print("\nFound the following relevant chunks:")
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



Searching for documents relevant to: 'What does apple sell??'

Found the following relevant chunks:

--- Result 1 ---
PART I — FINANCIAL INFORMATION
Item 1.    Financial Statements
Apple Inc.
CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)(In millions, except number of shares, which are reflected in thousands, and per-share amounts)
Three Months Ended Six Months Ended
March 29,2025 March 30,2024 March 29,2025 March 30,2024
Net sales:
   Products $ 68,714 $ 66,886 $ 166,674 $ 163,344 
   Services 26,645 23,867 52,985 46,984 
Total net sales 95,359 90,753 219,659 210,328 
Cost of sales:
   Products 44,030 42,424 103,477 100,864 
   Services 6,462 6,058 13,040 12,338 
Total cost of sales 50,492 48,482 116,517 113,202 
Gross margin 44,867 42,271 103,142 97,126 
Operating expenses:
Research and development 8,550 7,903 16,818 15,599 
Selling, general and administrative 6,728 6,468 13,903 13,254 
Total operating expenses 15,278 14,371 30,721 28,853 
Operating income 29,589 27,900

##### Again API issue

In [14]:
# from langchain import hub

# # N.B. for non-US LangSmith endpoints, you may need to specify
# # api_url="https://api.smith.langchain.com" in hub.pull.
# prompt = hub.pull("rlm/rag-prompt")

# example_messages = prompt.invoke(
#     {"context": "(context goes here)", "question": "(question goes here)"}
# ).to_messages()

# assert len(example_messages) == 1
# print(example_messages[0].content)



### Prompt Augmenting

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# This is the content of the "rlm/rag-prompt" created locally
# to avoid the SSL error from hub.pull().
RAG_PROMPT_TEMPLATE = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question}
Context: {context}
Answer:"""

prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

# Your example code will now work without a network call
example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: (question goes here)
Context: (context goes here)
Answer:


In [7]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [8]:
question = "What was Teslas revenue in the most recent quarter?"

retrieved_docs = vector_store.similarity_search(question)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

# Create a new variable for the invoked prompt, don't reassign 'prompt'
prepared_prompt = prompt.invoke({"question": question, "context": docs_content})

answer = llm.invoke(prepared_prompt) # Pass the prepared_prompt to the LLM

In [9]:
answer

AIMessage(content="According to the financial statements provided, Tesla's total revenue for the most recent quarter (Three Months Ended June 30, 2025) was $22,496 million. This figure includes automotive revenues, energy generation and storage, and services and other revenue streams.", additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '1ef2dc9a-fb39-4db8-9bac-9239fa2b6b81', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 30 Jul 2025 21:43:24 GMT', 'content-type': 'application/json', 'content-length': '452', 'connection': 'keep-alive', 'x-amzn-requestid': '1ef2dc9a-fb39-4db8-9bac-9239fa2b6b81'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [2997]}, 'model_name': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run--98a9d415-89ed-4b93-9258-c87571e02fc5-0', usage_metadata={'input_tokens': 1757, 'output_tokens': 59, 'total_tokens': 1816, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})


Searching for documents relevant to: 'What does apple sell??'

Found the following relevant chunks:

--- Result 1 ---
PART I — FINANCIAL INFORMATION
Item 1.    Financial Statements
Apple Inc.
CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)(In millions, except number of shares, which are reflected in thousands, and per-share amounts)
Three Months Ended Six Months Ended
March 29,2025 March 30,2024 March 29,2025 March 30,2024
Net sales:
   Products $ 68,714 $ 66,886 $ 166,674 $ 163,344 
   Services 26,645 23,867 52,985 46,984 
Total net sales 95,359 90,753 219,659 210,328 
Cost of sales:
   Products 44,030 42,424 103,477 100,864 
   Services 6,462 6,058 13,040 12,338 
Total cost of sales 50,492 48,482 116,517 113,202 
Gross margin 44,867 42,271 103,142 97,126 
Operating expenses:
Research and development 8,550 7,903 16,818 15,599 
Selling, general and administrative 6,728 6,468 13,903 13,254 
Total operating expenses 15,278 14,371 30,721 28,853 
Operating income 29,589 27,900

In [11]:
# my_prompt = input("Ask a question about the document: ")
my_prompt = "How much money does tesla make?"

print(f"\nSearching for documents relevant to: '{my_prompt}'")
# Use the vector store to find similar documents
# It automatically embeds the user's question and compares it to the chunk vectors.
results = vector_store.similarity_search(query=my_prompt,k=10)


retrieved_context = "\nFound the following relevant chunks from documents"

for i, doc in enumerate(results):
    retrieved_context+=(f"\n--- Result {i+1} ---")
    retrieved_context += f"\n{doc.page_content}" # Add a newline for readability
    # retrieved_context += f"\nMetadata: {doc.metadata}" # Convert metadata to string and add a label

augmented_prompt = "Answer this question: " + my_prompt + '\n Using this context \n' + retrieved_context 
print(augmented_prompt)


Searching for documents relevant to: 'How much money does tesla make?'
Answer this question: How much money does tesla make?
 Using this context 

Found the following relevant chunks from documents
--- Result 1 ---
Table	of	Contents
Tesla,	Inc.
Consolidated	Statements	of	Operations
(in	millions,	except	per	share	data)
(unaudited)
	
Three	Months	Ended	June	30,
Six	Months	Ended	June	30,
	
2025
2024
2025
2024
Revenues
Automotive	sales
$
15,787
	
$
18,530
	
$
28,712
	
$
34,990
	
Automotive	regulatory	credits
439
	
890
	
1,034
	
1,332
	
Automotive	leasing
435
	
458
	
882
	
934
	
Total	automotive	revenues
16,661
	
19,878
	
30,628
	
37,256
	
Energy	generation	and	storage
2,789
	
3,014
	
5,519
	
4,649
	
Services	and	other
3,046
	
2,608
	
5,684
	
4,896
	
Total	revenues
22,496
	
25,500
	
41,831
	
46,801
	
Cost	of	revenues
Automotive	sales
13,567
	
15,962
	
25,028
	
29,859
	
Automotive	leasing
228
	
245
	
467
	
514
	
Total	automotive	cost	of	revenues
13,795
	
16,207
	
25,495
	
30,373
	
Energy	ge

In [12]:
answer = llm.invoke(augmented_prompt)
answer

AIMessage(content="Based on the financial statements provided in the context, here are the key details about Tesla's revenue and income:\n\n- For the three months ended June 30, 2025:\n  - Total revenues: $22,496 million\n  - Net income: $1,190 million\n\n- For the six months ended June 30, 2025:\n  - Total revenues: $41,831 million  \n  - Net income: $1,610 million\n\nSo in the most recent quarter, Tesla made $22.5 billion in revenue and $1.2 billion in net income. For the first half of 2025, they made $41.8 billion in revenue and $1.6 billion in net income.\n\nThe financials show Tesla is generating significant revenue and profits, though exact amounts can vary quarter to quarter. Their automotive sales make up the majority of their revenue.", additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '2501704f-36c7-4ff5-bdd2-7366274360ac', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 30 Jul 2025 21:43:48 GMT', 'content-type': 'application/json', 'content-leng